In [1]:
pip install -U deepeval huggingface_hub --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.0/468.0 kB 7.9 MB/s eta 0:00:00


In [12]:
from deepeval.models.base_model import DeepEvalBaseLLM
from huggingface_hub import InferenceClient
from google.colab import userdata
api = userdata.get('HF_TOKEN')

class FinanceFlash(DeepEvalBaseLLM):

    def load_model(self):

        return InferenceClient(api_key=api)


    def generate(self, prompt: str) -> str:
        chat_model = self.load_model()
        resp = chat_model.chat.completions.create(
              model="Qwen/Qwen2.5-72B-Instruct",
  	          messages=[{'role':'user','content':prompt}],
  	          temperature=0.5,
  	          max_tokens=2048,
  	          top_p=0.7,

        )
        return resp.choices[0].message.content

    async def a_generate(self, prompt: str) -> str:
        return self.generate(prompt)

    def get_model_name(self):
        return "Custom Qwen Model"

In [14]:
from deepeval.metrics import GEval
from deepeval.test_case import LLMTestCaseParams


FinanceModel = FinanceFlash()
correctness_metric = GEval(
    name="Correctness",
    model = FinanceModel,
    criteria="Determine whether the actual output is factually correct. Act as a Financial Analyst who is judging outputs accordingly",
    # NOTE: you can only provide either criteria or evaluation_steps, and not both
    evaluation_steps=[
        "Look over the description for an idea of how the data looks.",
        "Go over the entire analysis and figure out if there are any factual discrepancies with reference to the description",
        "If something doesnt make sense financially speaking, penalize.",
        "You should also heavily penalize omission of detail",
        "Vague language, or contradicting OPINIONS, are not OK"
    ],
    evaluation_params=[LLMTestCaseParams.INPUT, LLMTestCaseParams.ACTUAL_OUTPUT],
)

In [17]:

# Need to read this off a file
rl_output = {
  "bond_yield_analysis": [
    {
      "bond_type": "US_10Y_Yield",
      "analysis": {
        "description": "This column contains both forecasted and actual 10-year US Treasury yields.",
        "trend": "Slight upward trend over time, ranging from approximately 2.43% to 2.94%",
        "volatility": "Low",
        "risk_level": "Low",
        "investment_implication": "Investors may prefer holding or cautiously considering buying opportunities given the stability of expected yields",
        "recommendation": "HOLD",
        "timing": "BUY NOW"
      }
    },
    {
      "bond_type": "AAA_Bond_Yield",
      "analysis": {
        "description": "This column includes forecasted and actual yields for AAA-rated bonds, which are considered less volatile due to their high credit rating.",
        "trend": "Moderate volatility, peaking around 5.76% and troughing at 4.09%",
        "volatility": "Moderate",
        "risk_level": "Moderate",
        "investment_implication": "Investors may need to balance their portfolios, possibly holding through uncertain times",
        "recommendation": "HOLD",
        "timing": "HOLD for 3 months"
      }
    },
    {
      "bond_type": "BAA_Bond_Yield",
      "analysis": {
        "description": "This column tracks the forecasted and actual yields for BAA-rated bonds, which are moderately volatile.",
        "trend": "Noticeable fluctuations, reaching up to nearly 6% and down to 4.40%",
        "volatility": "Medium",
        "risk_level": "Moderate",
        "investment_implication": "Fluctuations may require adjusting investment strategies periodically",
        "recommendation": "HOLD",
        "timing": "HOLD for 6 months"
      }
    },
    {
      "bond_type": "Junk_Bond_Yield",
      "analysis": {
        "description": "This column contains forecasted and actual yields for junk bonds, known for their high risk and higher yields.",
        "trend": "Significant volatility, with peaks approaching 6% and troughs at around 2.94%",
        "volatility": "High",
        "risk_level": "High",
        "investment_implication": "Investors should exercise caution, avoiding purchases unless expecting substantial gains",
        "recommendation": "SELL",
        "timing": "SELL NOW"
      }
    },
    {
      "bond_type": "US_10Y_Yield_actual",
      "analysis": {
        "description": "This column focuses solely on the actual 10-year US Treasury yields.",
        "trend": "Pronounced fluctuations, peaking around 4.22% and 4.63%",
        "volatility": "High",
        "risk_level": "High",
        "investment_implication": "Investors should be cautious due to the significant deviation from forecasted values",
        "recommendation": "SELL",
        "timing": "SELL NOW"
      }
    },
    {
      "bond_type": "AAA_Bond_Yield_actual",
      "analysis": {
        "description": "This column examines the actual yields for AAA-rated bonds.",
        "trend": "Consistent trends, with moderate ups and downs",
        "volatility": "Moderate",
        "risk_level": "Moderate",
        "investment_implication": "Investors may maintain their current positions, monitoring market changes",
        "recommendation": "HOLD",
        "timing": "HOLD for 3 months"
      }
    },
    {
      "bond_type": "BAA_Bond_Yield_actual",
      "analysis": {
        "description": "This column tracks the actual yields for BAA-rated bonds.",
        "trend": "Noticeable fluctuations, similar to the forecasted counterparts",
        "volatility": "Medium",
        "risk_level": "Moderate",
        "investment_implication": "Investors should stay flexible, adapting to market dynamics",
        "recommendation": "HOLD",
        "timing": "HOLD for 6 months"
      }
    },
    {
      "bond_type": "Junk_Bond_Yield_actual",
      "analysis": {
        "description": "This column focuses on the actual yields for junk bonds.",
        "trend": "Extreme fluctuations, including sharp increases and decreases",
        "volatility": "High",
        "risk_level": "High",
        "investment_implication": "Investors should approach these bonds with extreme caution, avoiding purchases unless gains are anticipated",
        "recommendation": "AVOID",
        "timing": "DO NOT TRADE"
      }
    }
  ]
}["bond_yield_analysis"]

In [18]:
gan_output = {
    "bond_yield_analysis": [
      {
        "bond_type": "US_10Y_Yield",
        "analysis": {
          "description": "This column represents the forecasted US 10-Year Treasury yield. Over time, the yield has shown a general upward trend, with fluctuations due to economic conditions such as inflation, Federal Reserve policies, and global events.",
          "trend": "The yield has increased from approximately 3.02% to 4.19%, indicating a strengthening economy or reduced inflation expectations.",
          "volatility": "Moderate",
          "risk_factors": "Interest rate risk and inflation risk",
          "investment_implication": "Investors may consider holding if the yield stabilizes or continues to rise, especially if expecting continued economic growth.",
          "recommendation": "HOLD",
          "timing": "HOLD NOW"
        }
      },
      {
        "bond_type": "US_10Y_Yield_actual",
        "analysis": {
          "description": "This column focuses solely on the actual 10-year US Treasury yields.",
          "trend": "Pronounced fluctuations, peaking around 4.22% and 4.63%",
          "volatility": "High",
          "risk_level": "High",
          "investment_implication": "Investors should be cautious due to the significant deviation from forecasted values",
          "recommendation": "SELL",
          "timing": "SELL NOW"
        }
      },
      {
        "bond_type": "AAA_Bond_Yield",
        "analysis": {
          "description": "This column represents the forecasted AAA bond yield. Over time, the yield has generally increased, though with some variability due to credit rating changes and economic conditions.",
          "trend": "The yield has risen from approximately 4.87% to 5.25%, indicating strong credit demand and economic confidence.",
          "volatility": "Moderate",
          "risk_factors": "Corporate profitability and default risks",
          "investment_implication": "Investors may consider buying if the yield is undervalued relative to longer-term yields.",
          "recommendation": "BUY",
          "timing": "BUY NOW"
        }
      },
      {
        "bond_type": "AAA_Bond_Yield_actual",
        "analysis": {
          "description": "This column examines the actual yields for AAA-rated bonds.",
          "trend": "Consistent trends, with moderate ups and downs",
          "volatility": "Moderate",
          "risk_level": "Moderate",
          "investment_implication": "Investors may maintain their current positions, monitoring market changes",
          "recommendation": "HOLD",
          "timing": "HOLD for 3 months"
        }
      },
      {
        "bond_type": "BAA_Bond_Yield",
        "analysis": {
          "description": "This column represents the forecasted BAA bond yield. Over time, the yield has shown moderate volatility, with ups and downs influenced by credit risk and economic factors.",
          "trend": "The yield has increased moderately, from approximately 5.09% to 5.82%, reflecting shifting credit conditions.",
          "volatility": "Significant",
          "risk_factors": "Credit risk and economic conditions",
          "investment_implication": "Buyers may consider entering if the yield is undervalued compared to AAA bonds.",
          "recommendation": "BUY",
          "timing": "BUY NOW"
        }
      },
      {
        "bond_type": "BAA_Bond_Yield_actual",
        "analysis": {
          "description": "This column tracks the actual yields for BAA-rated bonds.",
          "trend": "Noticeable fluctuations, similar to the forecasted counterparts",
          "volatility": "Medium",
          "risk_level": "Moderate",
          "investment_implication": "Investors should stay flexible, adapting to market dynamics",
          "recommendation": "HOLD",
          "timing": "HOLD for 6 months"
        }
      },
      {
        "bond_type": "Junk_Bond_Yield",
        "analysis": {
          "description": "This column represents the forecasted junk bond yield. Over time, the yield has shown significant volatility, driven by default risks and economic uncertainty.",
          "trend": "The yield has experienced sharp increases and decreases, ranging from 2.81% to 5.32%, reflecting high risk.",
          "volatility": "High",
          "risk_factors": "Default risks and investor sentiment toward speculative-grade bonds",
          "investment_implication": "Investors should exercise caution due to the high risk of default.",
          "recommendation": "SELL",
          "timing": "SELL NOW"
        }
      },
      {
        "bond_type": "Junk_Bond_Yield_actual",
        "analysis": {
          "description": "This column focuses on the actual yields for junk bonds.",
          "trend": "Extreme fluctuations, including sharp increases and decreases",
          "volatility": "High",
          "risk_level": "High",
          "investment_implication": "Investors should approach these bonds with extreme caution, avoiding purchases unless gains are anticipated",
          "recommendation": "AVOID",
          "timing": "DO NOT TRADE"
        }
      }
    ]
  }["bond_yield_analysis"]

In [28]:
rl_eval = []
from deepeval.test_case import LLMTestCase


for output in rl_output:
    test_case = LLMTestCase(
        input="""You are a senior financial advisor specializing in fixed income and bond markets, with over 20 years of experience. Your expertise covers analyzing market trends, forecasting yield movements, and evaluating risk factors.

Using the data provided below:
1. Forecasted bond yields for the next 3 months from two ARIMA models based off different datas
2. Historical bond yield data for the past 12 months

Please analyze these details thoroughly and offer clear investment recommendations. consider:
- Analysis of the forecasted trends
- Comparison with historical performance
- Consideration of yield volatility and risk-return trade-offs
""",
        actual_output=str(output),
    )
    correctness_metric.measure(test_case)
    rl_eval.append({"pred":output,"eval":correctness_metric.score,"reason":correctness_metric.reason})



Output()

Event loop is already running. Applying nest_asyncio patch to allow async execution...

Output()

Event loop is already running. Applying nest_asyncio patch to allow async execution...

Output()

...

Event loop is already running. Applying nest_asyncio patch to allow async execution...

Output()

Event loop is already running. Applying nest_asyncio patch to allow async execution...

Output()

Event loop is already running. Applying nest_asyncio patch to allow async execution...

In [29]:
gan_eval = []
from deepeval.test_case import LLMTestCase


for output in gan_output:
    test_case = LLMTestCase(
        input="""You are a senior financial advisor specializing in fixed income and bond markets, with over 20 years of experience. Your expertise covers analyzing market trends, forecasting yield movements, and evaluating risk factors.

Using the data provided below:
1. Forecasted bond yields for the next 3 months from two ARIMA models based off different datas
2. Historical bond yield data for the past 12 months

Please analyze these details thoroughly and offer clear investment recommendations for each of the four bonds (AAA, BAA, US10Y, and Junk). For each bond, consider:
- Analysis of the forecasted trends
- Comparison with historical performance
- Consideration of yield volatility and risk-return trade-offs
""",
        actual_output=str(output),
    )
    correctness_metric.measure(test_case)
    gan_eval.append({"pred":output,"eval":correctness_metric.score,"reason":correctness_metric.reason})



Output()

Event loop is already running. Applying nest_asyncio patch to allow async execution...

Output()

Event loop is already running. Applying nest_asyncio patch to allow async execution...

Output()

...

Event loop is already running. Applying nest_asyncio patch to allow async execution...

Output()

Event loop is already running. Applying nest_asyncio patch to allow async execution...

Output()

Event loop is already running. Applying nest_asyncio patch to allow async execution...

In [30]:
rl_eval

[{'pred': {'bond_type': 'US_10Y_Yield',
   'analysis': {'description': 'This column contains both forecasted and actual 10-year US Treasury yields.',
    'trend': 'Slight upward trend over time, ranging from approximately 2.43% to 2.94%',
    'volatility': 'Low',
    'risk_level': 'Low',
    'investment_implication': 'Investors may prefer holding or cautiously considering buying opportunities given the stability of expected yields',
    'recommendation': 'HOLD',
    'timing': 'BUY NOW'}},
  'eval': 0.4,
  'reason': 'The analysis provides a basic trend and recommendation but lacks detailed comparison with historical data, omits specific forecasted yields from the two ARIMA models, and does not thoroughly evaluate yield volatility and risk-return trade-offs. The recommendation is also vague and does not align with the provided data.'},
 {'pred': {'bond_type': 'AAA_Bond_Yield',
   'analysis': {'description': 'This column includes forecasted and actual yields for AAA-rated bonds, which are

In [31]:
gan_eval

[{'pred': {'bond_type': 'US_10Y_Yield',
   'analysis': {'description': 'This column represents the forecasted US 10-Year Treasury yield. Over time, the yield has shown a general upward trend, with fluctuations due to economic conditions such as inflation, Federal Reserve policies, and global events.',
    'trend': 'The yield has increased from approximately 3.02% to 4.19%, indicating a strengthening economy or reduced inflation expectations.',
    'volatility': 'Moderate',
    'risk_factors': 'Interest rate risk and inflation risk',
    'investment_implication': 'Investors may consider holding if the yield stabilizes or continues to rise, especially if expecting continued economic growth.',
    'recommendation': 'HOLD',
    'timing': 'HOLD NOW'}},
  'eval': 0.4,
  'reason': 'The analysis only covers the US10Y bond, omitting AAA, BAA, and Junk bonds. It provides a trend and recommendation but lacks a detailed comparison with historical performance, consideration of yield volatility, and